# Run alignment and variant calling

The parameters we use are:
    * nanopolish: 5-mer alignment, 6-mer variant calling, reverse HMM fix

In [1]:
ROOTDIR="/mnt/ebola/ebov"
import os
os.environ['PATH'] = os.getenv('PATH') + ':' + ROOTDIR + '/scripts' + ':' + ROOTDIR + '/bin/standard-RAxML' + ':' + '/bin/nanopore-scripts'

In [2]:
%cd $ROOTDIR

/mnt/ebola/ebov


In [5]:
%%bash

ROOTDIR="/mnt/ebola/ebov"
cd $ROOTDIR
export PATH=`pwd`/bin/bwa:$PATH
export PATH=$PATH:`pwd`/scripts
export PATH=$PATH:`pwd`/bin/nanopore-scripts
export PATH=$PATH:`pwd`/bin/samtools
export PATH=$PATH:`pwd`/bin/marginAlign

cd analysis-EMLab
makecommands.py ../metadata/metadata.db EMLab qual.sh | parallel -j40


+ ref_prefix=EM_079517
+ sample=FCH_2015_209B
+ poretools_dir=FCH_2015_209B
+ sample_tag=FCH_2015_209B_hq
+ second_batch=na
+ read_type=hq
+ bwa mem -t 1 -x ont2d ../refs/EM_079517.fasta FCH_2015_209B_hq.fasta
+ samtools view -bS -
+ samtools sort - EM_079517_FCH_2015_209B_hq_bwa.sorted
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[bam_header_read] EOF marker is absent. The input is probably truncated.
[M::process] read 251 sequences (753447 bp)...
[M::mem_process_seqs] Processed 251 reads in 5.692 CPU sec, 5.692 real sec
[samopen] SAM header is present: 1 sequences.
[main] Version: 0.7.12-r1044
[main] CMD: bwa mem -t 1 -x ont2d ../refs/EM_079517.fasta FCH_2015_209B_hq.fasta
[main] Real time: 6.361 sec; CPU: 6.152 sec
+ expand-cigar.py --fasta ../refs/EM_079517.fasta --bam EM_079517_FCH_2015_209B_hq_bwa.sorted.bam
+ count-errors.py -
+ expand-cigar.py --fasta ../refs/EM_079517.fasta --bam EM_079517_FCH_2015_209B_hq_marginalign.sorted.bam
+ count-errors.py -
+ ref_prefix=EM_079517_mut

In [ ]:
## After the raw files are produced, show statistics.

%cd analysis-var6mer-aln5mer-reverse-marginalign-all

In [ ]:
!makecommands.py ../metadata/metadata.db EMLab consensus.sh | parallel -j40

In [ ]:
!dump_consensus.sh ../metadata/metadata.db 20151028

In [ ]:
!rm *oct28*
!make -f ../scripts/tree.mk PREFIX=oct28 FASTA=consensus-20151028-tpr0.75.tagged.fasta THREADS=40 DIST=0.0002

In [ ]:
!make_stats_file.py ../metadata/metadata.db EMLab | intersection_vcf_stats.py /dev/stdin > tp_stats.txt

In [ ]:
!head -10 tp_stats.txt

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
stats=read.table("tp_stats.txt", header=T, sep="\t")
a = nrow(subset(stats, tag == 'np-new-filter_qual200-50' & TPR > 0.8))
subset(stats, tag == 'np-new-filter_qual200-50' & TPR < 0.2)


In [ ]:
%%R

library(ggplot2)
stats=read.table("tp_stats.txt", header=T, sep="\t")
ggplot(stats, aes(x=TPR)) + geom_histogram() + facet_wrap(~tag)